<a href="https://colab.research.google.com/github/Neel7317/Deep-Learning-/blob/master/Bidirectional_LSTM_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fake News Classifier using Bidirectional LSTM

In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR']="/content"

In [ ]:
!kaggle competitions download -c fake-news

In [4]:
import pandas as pd

In [7]:
df=pd.read_csv('train.csv.zip')

In [9]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
###Drop Nan Values
df=df.dropna()

In [11]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [12]:
## Get the Dependent features
y=df['label']

In [13]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [14]:
X.shape

(18285, 4)

In [15]:
y.shape

(18285,)

In [16]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [17]:
### Vocabulary size
voc_size=5000

One hot representation

In [18]:
messages=X.copy()

In [19]:
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [20]:
messages.reset_index(inplace=True)

In [21]:
import nltk
import re
from nltk.corpus import stopwords

In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:

### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [57]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [58]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[0]

[3552, 2842, 3140, 255, 412, 360, 1637, 1811, 2128, 2448]

Embedding Representation

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

In [27]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3552,
       2842, 3140,  255,  412,  360, 1637, 1811, 2128, 2448], dtype=int32)

In [28]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [30]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [31]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)


Model Training

In [32]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 12s 15ms/step - loss: 0.3122 - accuracy: 0.8522 - val_loss: 0.1965 - val_accuracy: 0.9153
Epoch 2/10
192/192 [==============================] - 2s 10ms/step - loss: 0.1414 - accuracy: 0.9451 - val_loss: 0.2179 - val_accuracy: 0.9186
Epoch 3/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0958 - accuracy: 0.9637 - val_loss: 0.2287 - val_accuracy: 0.9140
Epoch 4/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0679 - accuracy: 0.9755 - val_loss: 0.3145 - val_accuracy: 0.9118
Epoch 5/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0492 - accuracy: 0.9829 - val_loss: 0.3173 - val_accuracy: 0.9092
Epoch 6/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0356 - accuracy: 0.9891 - val_loss: 0.3525 - val_accuracy: 0.9085
Epoch 7/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0266 - accuracy: 0.9918 - val_loss: 0.4598 - val_accuracy: 0.907

In [54]:
X_train[0],y_train[0]

(array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0, 1037], dtype=int32), 1)

In [43]:
y_pred1=model1.predict_classes(X_train[3])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '



Performance Metrics And Accuracy

In [55]:
y_pred=model1.predict_classes([[  0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0, 1037]])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [56]:
y_pred

array([[1]], dtype=int32)

In [46]:
from sklearn.metrics import confusion_matrix

In [48]:
confusion_matrix(y_test,y_pred)

array([[3053,  366],
       [ 196, 2420]])

In [49]:

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9068765534382767

In [51]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.89      0.92      3419
           1       0.87      0.93      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.90      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

